# Choosing the correct value function for ECRH/q optimisation

### Profiles for comparison

Load data from the SPR45 benchmark (current state-of-the-art solution).

In [1]:
from jetto_tools.results import JettoResults

spr45 = JettoResults(path="../data/benchmark/")
spr45_profiles = spr45.load_profiles()
spr45_timetraces = spr45.load_timetraces()

16:36:21 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14


Load data from example BayesOpt run:

In [2]:
import netCDF4

path = "../data/piecewise_linear_50/bayesopt/3/5"
pl50_3_5_profiles = netCDF4.Dataset(f"{path}/profiles.CDF")
pl50_3_5_timetraces = netCDF4.Dataset(f"{path}/timetraces.CDF")


In [3]:
path = "../data/piecewise_linear_50/bayesopt/9/19"
pl50_9_19_profiles = netCDF4.Dataset(f"{path}/profiles.CDF")
pl50_9_19_timetraces = netCDF4.Dataset(f"{path}/timetraces.CDF")


Compare the solutions:

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

colours = plotly.colors.qualitative.Vivid

figure = make_subplots(1, 2,
                       shared_xaxes=True)
i = 0
for solution_name, profiles in {'SPR45': spr45_profiles,
                        'PL50-3.5': pl50_3_5_profiles,
                        'PL50-9.19': pl50_9_19_profiles}.items():
    figure.add_traces(
        [
            go.Scatter(
                x=profiles["XRHO"][-1],
                y=profiles["QECE"][-1],
                line_color=colours[i],
                legendgroup=i,
                name=solution_name,
            ),
            go.Scatter(
                x=profiles["XRHO"][-1],
                y=profiles["Q"][-1],
                line_color=colours[i],
                showlegend=False,
                legendgroup=i,
            ),
        ],
        rows = [1, 1],
        cols = [1, 2]
    )
    i += 1
    
figure.update_layout(template="simple_white")
figure.update_xaxes(title_text="Normalised radius")
figure.update_yaxes(title_text="ECRH power density", row=1, col=1)
figure.update_yaxes(title_text="Safety factor", row=1, col=2)
figure.show()

In [5]:
from prettytable import PrettyTable

property_table = PrettyTable(["Property", "SPR45", "PL50-3.5", "PL50-9.19"])
property_table.float_format = ".2"

property_table.add_row(["Q0",
                        spr45_timetraces["Q0"][-1].data,
                        pl50_3_5_timetraces["Q0"][-1].data,
                        pl50_9_19_timetraces["Q0"][-1].data])
property_table.add_row(["QMIN",
                        spr45_timetraces["QMIN"][-1].data,
                        pl50_3_5_timetraces["QMIN"][-1].data,
                        pl50_9_19_timetraces["QMIN"][-1].data])
property_table.add_row(["ROQM",
                        spr45_timetraces["ROQM"][-1].data,
                        pl50_3_5_timetraces["ROQM"][-1].data,
                        pl50_9_19_timetraces["ROQM"][-1].data])
property_table

Property,SPR45,PL50-3.5,PL50-9.19
Q0,4.108164,2.9542603,2.343364
QMIN,2.256152,2.1861162,2.1784601
ROQM,0.23076923,0.050167225,0.016722407


### Genetic algorithm cost function

This cost function is taken directly from the `smars-develop` branch of jetto-pythontools.

In [6]:
from jetto_mobo import genetic_algorithm

Compare the value of the loaded solutions:

In [7]:
t = PrettyTable(['Solution', 'GA value'])
t.float_format = ".2"
t.add_row(["SPR45", genetic_algorithm.scalar_objective(spr45_profiles, spr45_timetraces)])
t.add_row(["PL50-3.5", genetic_algorithm.scalar_objective(pl50_3_5_profiles, pl50_3_5_timetraces)])
t.add_row(["PL50-9.19", genetic_algorithm.scalar_objective(pl50_9_19_profiles, pl50_9_19_timetraces)])
t

Solution,GA value
SPR45,0.91
PL50-3.5,0.85
PL50-9.19,0.79


### Understanding the value function

In [8]:
from typing import Iterable, Callable
import numpy as np 

def multi_objective_plot(labels: Iterable[str], objective_function: Callable[[netCDF4.Dataset, netCDF4.Dataset], np.ndarray]):
    figure = go.Figure()
    figure.add_traces(
        [
            go.Scatterpolar(
                r=objective_function(spr45_profiles, spr45_timetraces),
                theta=labels,
                fill="toself",
                line_color=colours[0],
                name="SPR45"
            ), 
            go.Scatterpolar(
                r=objective_function(pl50_3_5_profiles, pl50_3_5_timetraces),
                theta=labels,
                fill="toself",
                line_color=colours[1],
                name="PL50-3.5"
            ),
            go.Scatterpolar(
                r=objective_function(pl50_9_19_profiles, pl50_9_19_timetraces),
                theta=labels,
                fill="toself",
                line_color=colours[2],
                name="PL50-9.19"
            )
        ]
    )
    figure.update_layout(template="seaborn")
    return figure

In [9]:
multi_objective_plot(labels=["|q(0) - min(q)|",
                             "|min(q) - 2.2|",
                             "|argmin(q)|",
                             "Non-monotonic fraction of q",
                             "Non-monotonic fraction of dq",
                             "rho of q=3",
                             "rho of q=4"],
                     objective_function=genetic_algorithm.vector_objective)

## Normalised objective function (draft)

Same mathematical objectives, now formulated as functions to maximise. All objectives are normalised to [0, 1].

In [10]:
from jetto_mobo import objective as bo_objective

In [11]:
from copy import deepcopy
# Add to table
t2 = deepcopy(t)
t2.add_column("MOBO objective", 
             [bo_objective.scalar_objective(spr45_profiles, spr45_timetraces),
              bo_objective.scalar_objective(pl50_3_5_profiles, pl50_3_5_timetraces),
              bo_objective.scalar_objective(pl50_9_19_profiles, pl50_9_19_timetraces)])
t2.float_format = ".2"
figure = multi_objective_plot(labels=["exp(-| q(0) - min(q) |)",
                             "exp(-| min(q) - 2.2 |)",
                             "1 - argmin(q)",
                             "Area fraction of increasing q",
                             "Area fraction of increasing dq",
                             "rho at q=3",
                             "rho at q=4"],
                     objective_function=bo_objective.vector_objective)
# figure.update_polars(radialaxis_range=[0, 1.01])
figure.show()
t2

Solution,GA value,MOBO objective
SPR45,0.91,0.68
PL50-3.5,0.85,0.75
PL50-9.19,0.79,0.65


Note that the order is different from the GA - PL50-2.5 beats SPR45 considerably.

Add weights to the vector objective:

In [13]:
weights = np.array([1, 1, 1, 5, 5, 1, 1])

# Add to table
t3 = deepcopy(t2)
t3.add_column("Weighted MOBO objective", 
             [bo_objective.scalar_objective(spr45_profiles, spr45_timetraces, weights),
              bo_objective.scalar_objective(pl50_3_5_profiles, pl50_3_5_timetraces, weights),
              bo_objective.scalar_objective(pl50_9_19_profiles, pl50_9_19_timetraces, weights)])
t3.float_format = ".3"

figure = multi_objective_plot(labels=["exp(-| q(0) - min(q) |)",
                             "exp(-| min(q) - 2.2 |)",
                             "1 - argmin(q)",
                             "Area fraction of increasing q",
                             "Area fraction of increasing dq",
                             "rho at q=3",
                             "rho at q=4"],
                     objective_function=lambda p,t: weights * bo_objective.vector_objective(p, t))
# figure.update_polars(radialaxis_range=[0, 1.01])
figure.show()
t3

Solution,GA value,MOBO objective,Weighted MOBO objective
SPR45,0.910,0.679,1.696
PL50-3.5,0.852,0.748,1.750
PL50-9.19,0.788,0.649,1.572
